In [1]:
import math
import pandas as pd

In [2]:
df = pd.read_csv("../data/babyNamesUSYOB-full.csv")

In [3]:
grouped_by_sex = df.groupby(["YearOfBirth", "Sex"])["Number"].sum().reset_index()

In [4]:
def add_both(x):
    last_row = x.iloc[-1].copy()
    last_row["Sex"] = "Both"
    last_row["Number"] = int(x[x["Sex"] == "M"]["Number"]) + int(x[x["Sex"] == "F"]["Number"])
    return x.append(last_row)
total_births = grouped_by_sex.groupby("YearOfBirth").apply(add_both).reset_index(drop=True)

In [5]:
total_births.to_csv("../data/total_births.csv", index=False)

In [6]:
unique_names = df.groupby(["YearOfBirth", "Sex"])["Name"].count().reset_index()

In [7]:
unique_names_both = df.drop(["Sex", "Number"], axis=1).drop_duplicates().groupby("YearOfBirth")["Name"].count().reset_index()

In [8]:
def add_both_unique(x):
    last_row = x.iloc[-1].copy()
    last_row["Sex"] = "Both"
    last_row["Name"] = int(unique_names_both[unique_names_both["YearOfBirth"] == last_row["YearOfBirth"]]["Name"])
    return x.append(last_row)
unique_names = unique_names.groupby("YearOfBirth").apply(add_both_unique).reset_index(drop=True)

In [9]:
unique_names = unique_names.rename(columns={"Name":"Count"})

In [10]:
unique_names.to_csv("../data/unique_names.csv", index=False)

In [11]:
def add_prop(group):
    group["Proportion"] = group.Number / group.Number.sum()
    return group

name_records = df.groupby(["YearOfBirth","Sex"]).apply(add_prop)

In [12]:
name_records["NameLength"] = name_records["Name"].apply(lambda x: len(x))

In [13]:
name_records["Decade"] = name_records["YearOfBirth"].apply(lambda x: math.floor(x/10)*10)
name_records.head()

,YearOfBirth,Name,Sex,Number,Proportion,NameLength,Decade
0,1880,Mary,F,7065,0.077644,4,1880
1,1880,Anna,F,2604,0.028618,4,1880
2,1880,Emma,F,2003,0.022013,4,1880
3,1880,Elizabeth,F,1939,0.021310,9,1880
4,1880,Minnie,F,1746,0.019189,6,1880


In [14]:
name_records.to_csv("../data/name_records.csv", index=False)

In [15]:
def select_top_10(group):
    group = group.sort_values(by=["Number"], ascending=False).reset_index(drop=True)[:10]
    group["Rank"] = [i for i in range(1, 11)]
    return group
name_rankings = name_records.groupby(["Decade", "Name"])["Number"].mean().reset_index().groupby("Decade").apply(select_top_10).reset_index(drop=True)

In [16]:
name_rankings.to_csv("../data/name_rankings.csv", index=False)

In [17]:
decades = name_rankings["Decade"].unique()
def add_missing_ranks(group):
    for decade in decades:
        if group[group["Decade"] == decade].size == 0:
            last_row = group.iloc[-1].copy()
            last_row["Decade"] = decade
            last_row["Rank"] = 12
            last_row["Number"] = 0
            group = group.append(last_row)
    return group

name_rankings_with_11 = name_rankings.groupby("Name").apply(add_missing_ranks).reset_index(drop=True).sort_values(by=["Decade"]).reset_index(drop=True)

In [18]:
name_rankings_with_11.to_csv("../data/name_rankings_with_11.csv", index=False)